In [4]:

import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import re
import os
from scipy.spatial.transform import Rotation as R


In [16]:
import math
 
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians


In [17]:
def parse_data(PATH):
    files = glob.glob(f'{PATH}*[0-9].txt')
    files = sorted(files, key=lambda x:int(re.findall("(\d+)",x)[-1]))

    poses = [None]*len(files)
    coord = []
    angles = []


    for i, f in enumerate(files):
        
    #print(np.loadtxt(i))
        with open(f) as f_input:
            
            lines = f_input.readlines()
            
        poses[i] = []
        for m in lines:
            m = re.sub(r'\[|\]' ,'', m)
            m = [float(s) for s in m.split()]
            poses[i] = poses[i] + m

        
    for i in range(len(poses)):
        coord.append((poses[i][0], poses[i][1], poses[i][2]))
        

    for i in range(len(poses)):
        angles.append((poses[i][3], poses[i][4], poses[i][5]))

    return angles, coord


In [29]:
def get_rot(Q):
    # R = np.array(angles)
    # R, J = cv2.Rodrigues(R)
    w = np.sqrt(1.0 - Q[0]**2 - Q[1]**2 - Q[2]**2)
    
    angles = euler_from_quaternion(Q[0], Q[1], Q[2], w)
    Rx = [[1, 0, 0], [0, np.cos(angles[0]), -np.sin(angles[0])], [0, np.sin(angles[0]), np.cos(angles[0])]]
    Ry = [[np.cos(angles[1]), 0, np.sin(angles[1])], [0, 1, 0], [-np.sin(angles[1]), 0, np.cos(angles[1])]]
    Rz = [[np.cos(angles[2]), -np.sin(angles[2]), 0], [np.sin(angles[2]), np.cos(angles[2]), 0], [0, 0, 1]]
    R = np.matmul(Rz, np.matmul(Ry, Rx))
    return R

def get_translation(coord):
    z = coord[2]
    x = coord[0]
    y = coord[1]

    offset_vicon_to_base = np.array([108, 0, -13.4])
    offset_base_to_rgb = np.array([10, 32, 13])
    #From vicon link to base link
    
    T = np.array([coord[0], coord[1], coord[2]])
    T = T + offset_vicon_to_base
    T = T + offset_base_to_rgb
    T = T*10**(-3)
    return T

def get_intrinsic(fx, fy, cx, cy):
    intrin = np.array([[fx, 0 , cx],
                       [0, fy, cy],
                       [0, 0, 1]])

    return intrin

In [30]:
def main():
    
    transform = np.zeros((4,4))

    fx = 615.671 # focal dist of cam lense [mm]
    fy = 615.962 # 
    cx = 328.001 # optical center
    cy = 241.31

    outdir = '/home/iana/Atlas/drone_view/'
    indir = '/home/iana/Atlas/drone_view/poses_raw/'

    angles = parse_data(indir)[0] # get raw angle data
    coord = parse_data(indir)[1] # get raw coord
    
    t1_disp = 0
    t2_disp = 90
    t3_disp = -90

    tx = -90*(np.pi/180) # -90
    ty = 0.776*(np.pi/180) #0.776
    tz = -90*(np.pi/180) # -90
    Rx = [[1, 0, 0], [0, np.cos(tx), -np.sin(tx)], [0, np.sin(tx), np.cos(tx)]]
    Ry = [[np.cos(ty), 0, np.sin(ty)], [0, 1, 0], [-np.sin(ty), 0, np.cos(ty)]]
    Rz = [[np.cos(tz), -np.sin(tz), 0], [np.sin(tz), np.cos(tz), 0], [0, 0, 1]]
    #R_base_to_camera_color_opt = get_rot((tx, ty, tz))
    
    for i, (ang, c) in enumerate(zip(angles, coord)):
    
        R = get_rot(ang)
        R = np.matmul(Rz, np.matmul(Ry, np.matmul(Rx, R)))
        #R = R_base_to_camera_color_opt @ R

        transform[0:3, 0:3] = R
       

        T = get_translation(c)
        transform[0:3, 3] = T
        transform[3, 3] = 1
        np.savetxt(f'{outdir}pose/{i+1:08}.txt', transform)
    
    cam_param= get_intrinsic(fx, fy, cx, cy)
    np.savetxt(f'{outdir}intrinsics.txt', cam_param)

main()

In [ ]:
outdir = '/home/iana/Atlas/drone_view/color/'


for i, f in enumerate(os.listdir(outdir)):

    old_name = f'{outdir}{i+1:07}.jpg'
    new_name = f'{outdir}{i+1:08}.jpg'

    # Renaming the file
    os.rename(old_name, new_name)